
# 漢字・アルファベットからカタカナを推測するモデルを学習する
オープンソース辞書及び，Wikipediaから作成した漢字とカタカナのペアデータを用いて，Transformer+Seq2Seqのモデルで，スクラッチで学習する。

入力の文字（漢字・アルファベットなど）は一文字ずつに分割，また，出力の文字（カタカナ）も一文字ずつに分割して，学習する。



## データ準備
データは，[dict](../dict)で作成した姓名辞書データのoss.json及びseimei.jsonと，Wikipediaから抜き出した，漢字姓名とカタカナ姓名（スペース区切りのもの）を用いて，学習に使用した。単語単位である姓名辞書データと，Wikipediaのスペースで区切られた姓名を用いることで，姓名で漢字とカナが入力された場合の学習を可能とした。

また，ひらがなとカタカナ，


In [1]:
# スクリプトで使う環境変数をセット
import os
os.environ['DIR']='dataset'
os.environ['MODEL']='model'
os.environ["LOG"]='logs'
os.environ['RATIO']="0.01"
os.environ['WIKI']="../dict/wikipedia/wikiname.txt" # Wikipediaの概要欄から取得した姓名の漢字・アルファベットとカタカナのペアデータ。このファイルの姓名をスペースで区切って丹後他院位にしたものは，辞書ファイルに追加済み

os.makedirs(os.environ['DIR'],exist_ok=True)
os.makedirs(os.environ['MODEL'],exist_ok=True)

In [2]:
# オープンソース辞書とWikipediaから作成した単語辞書をマージする
!python mergedic.py --dic1 ../dict/oss.json --dic2 ../dict/seimei.json --outfile tmp.json



{
  "dic1": "../dict/oss.json",
  "dic2": "../dict/seimei.json",
  "outfile": "tmp.json"
}
len=2150171
cnt=2252554


In [3]:
# 作成したデータを訓練用，開発用，検証用に分ける。開発用と検証用がそれぞれ全体の0.01
!python prep.py --json tmp.json --outdir $DIR --ratio $RATIO


{
  "json": "tmp.json",
  "outdir": "dataset",
  "ratio": 0.01,
  "reverse": false
}
k_max_len=72
v_max_len=120
train,2207631
valid,22526
test,22526


In [4]:
# wikipediaの漢字姓名とカナ姓名を取得し，訓練，開発，検証用に分けて，辞書から作成したデータに追加する。
!python catawk.py --begin_idx_per 0 --end_idx_per 1 --appendfile $DIR/valid.src --index 0 --infile $WIKI --infile_delimiter tsv
!python catawk.py --begin_idx_per 0 --end_idx_per 1 --appendfile $DIR/valid.tgt --index 1 --infile $WIKI --infile_delimiter tsv
!python catawk.py --begin_idx_per 1 --end_idx_per 2 --appendfile $DIR/test.src --index 0 --infile $WIKI --infile_delimiter tsv
!python catawk.py --begin_idx_per 1 --end_idx_per 2 --appendfile $DIR/test.tgt --index 1 --infile $WIKI --infile_delimiter tsv
!python catawk.py --begin_idx_per 2 --end_idx_per 100 --appendfile $DIR/train.src --index 0 --infile $WIKI --infile_delimiter tsv
!python catawk.py --begin_idx_per 2 --end_idx_per 100 --appendfile $DIR/train.tgt --index 1 --infile $WIKI --infile_delimiter tsv

Traceback (most recent call last):
  File "/home/analysis01/src/kanjikana-model/train/catawk.py", line 46, in <module>
    main()
  File "/home/analysis01/src/kanjikana-model/train/catawk.py", line 43, in main
    run(args)
  File "/home/analysis01/src/kanjikana-model/train/catawk.py", line 20, in run
    with open(args.infile,'r',encoding='utf-8') as f:
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
FileNotFoundError: [Errno 2] No such file or directory: '../dict/wikipedia/wikiname.txt'
Traceback (most recent call last):
  File "/home/analysis01/src/kanjikana-model/train/catawk.py", line 46, in <module>
    main()
  File "/home/analysis01/src/kanjikana-model/train/catawk.py", line 43, in main
    run(args)
  File "/home/analysis01/src/kanjikana-model/train/catawk.py", line 20, in run
    with open(args.infile,'r',encoding='utf-8') as f:
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
FileNotFoundError: [Errno 2] No such file or directory: '../dict/wikipedia/wikiname.txt'
Traceback (m

In [5]:
# ひらがなとカタカナのペアデータを作成し，訓練データに混ぜる
!python kana.py --outfile $DIR/hirakata.txt

!python catawk.py --begin_idx_per 0 --end_idx_per 100 --appendfile $DIR/train.src --index 0 --infile $DIR/hirakata.txt
!python catawk.py --begin_idx_per 0 --end_idx_per 100 --appendfile $DIR/train.tgt --index 1 --infile $DIR/hirakata.txt

{
  "outfile": "dataset/hirakata.txt",
  "reverse": false
}


In [6]:
# 単漢字とカタカナのデータを，訓練データに混ぜる 
!python jsontotext.py --jsonfile ../dict/tankanji.json --outfile tankanji.txt

!python catawk.py --begin_idx_per 0 --end_idx_per 100 --appendfile $DIR/train.src --index 0 --infile tankanji.txt
!python catawk.py --begin_idx_per 0 --end_idx_per 100 --appendfile $DIR/train.tgt --index 1 --infile tankanji.txt

{
  "jsonfile": "../dict/tankanji.json",
  "outfile": "tankanji.txt",
  "reverse": false
}


In [7]:
# 開発，検証用に，訓練データに入っているものがあれば，削除する(インサンプルを阻止)
!python omit_testval.py --train_src $DIR/train.src --train_tgt $DIR/train.tgt --test_src $DIR/test.src --test_tgt $DIR/test.tgt --valid_src $DIR/valid.src --valid_tgt $DIR/valid.tgt



{
  "train_src": "dataset/train.src",
  "train_tgt": "dataset/train.tgt",
  "test_src": "dataset/test.src",
  "test_tgt": "dataset/test.tgt",
  "valid_src": "dataset/valid.src",
  "valid_tgt": "dataset/valid.tgt"
}
train=2325795
omit=225


In [8]:
# 訓練，開発，検証データをシャッフルする

!python shuffle.py --src $DIR/train.src --tgt $DIR/train.tgt
!python shuffle.py --src $DIR/valid.src --tgt $DIR/valid.tgt
!python shuffle.py --src $DIR/test.src --tgt $DIR/test.tgt



{
  "tgt": "dataset/train.tgt",
  "src": "dataset/train.src"
}
{
  "tgt": "dataset/valid.tgt",
  "src": "dataset/valid.src"
}
{
  "tgt": "dataset/test.tgt",
  "src": "dataset/test.src"
}


In [9]:
# データをスペース区切りにする（文字単位で，学習させるため）

!python space.py --infile $DIR/train.src --outfile $DIR/train.src
!python space.py --infile $DIR/train.tgt --outfile $DIR/train.tgt
!python space.py --infile $DIR/valid.src --outfile $DIR/valid.src
!python space.py --infile $DIR/valid.tgt --outfile $DIR/valid.tgt
!python space.py --infile $DIR/test.src --outfile $DIR/test.src
!python space.py --infile $DIR/test.tgt --outfile $DIR/test.tgt



{
  "infile": "dataset/train.src",
  "outfile": "dataset/train.src"
}
{
  "infile": "dataset/train.tgt",
  "outfile": "dataset/train.tgt"
}
{
  "infile": "dataset/valid.src",
  "outfile": "dataset/valid.src"
}
{
  "infile": "dataset/valid.tgt",
  "outfile": "dataset/valid.tgt"
}
{
  "infile": "dataset/test.src",
  "outfile": "dataset/test.src"
}
{
  "infile": "dataset/test.tgt",
  "outfile": "dataset/test.tgt"
}


In [10]:
# jsonl形式に変換する

!python format.py --src $DIR/train.src --tgt $DIR/train.tgt --outfile $DIR/train.jsonl --src_key kanji --tgt_key kana
!python format.py --src $DIR/test.src --tgt $DIR/test.tgt --outfile $DIR/test.jsonl --src_key kanji --tgt_key kana
!python format.py --src $DIR/valid.src --tgt $DIR/valid.tgt --outfile $DIR/valid.jsonl --src_key kanji --tgt_key kana

## 漢字・アルファベットとカタカナのペアから学習する
学習は，訓練用データを用いて学習し，１エポックごとに開発用データ(valid.json)でLossを確認する。


### パラメタ
- emb_size 
入力，出力の文字のエンベッドのDimension

- nhead    
マルチヘッド数

- ffn_hid_dim    
FFNのDimension

- num_encoder_layer    
エンコードレイヤーの数

- num_decoder_layer    
デコードレイヤーの数

- lr     
学習率

- dropout    
ドロップアウトの割合, 0-1

- num_epochs    
何周学習データを用いて学習を行うか

- device    
mps,cpu,cudaから選択
  - mps        
   アップルシリコンを搭載したマシンで実行する際に選択
  - cuda         
   CUDAが利用できる環境で選択
  - cpu        
   上記以外は，CPUモードを選択

- earlystop_patient    
開発用データでlossが下がらなくなってearlystop_patient回計算が終了した場合に，num_epochs数以下でも，計算を終了させる

- output_dir    
モデルの出力ディレクトリ

- tensorboard_logdir    
tensorboard形式のログの出力ディレクトリ。学習状況を確認するためには`tensorboard --logdir logs`を実行後，ブラウザでhttp://localhost:6000/から確認
 



- prefix    
jsonl形式のデータのprefix

- source_lang    
jsonl形式のデータのsourceのキー

- target_lang    
jsonl形式のデータのtargetのキー

- train_file    
訓練用データのJSONLファイル

- valid_file    
開発用データのJSONLファイル

In [13]:
# 訓練実行

!python ./char_model.py \
  --emb_size 512 \
  --nhead 8 \
  --ffn_hid_dim 2048 \
  --batch_size 128 \
  --num_encoder_layers 8 \
  --num_decoder_layers 8 \
  --lr 0.00002 \
  --dropout 0.3 \
  --num_epochs 50 \
  --device mps \
  --earlystop_patient 3 \
  --output_dir $MODEL \
  --tensorboard_logdir $LOG \
  --prefix translation \
  --source_lang kanji \
  --target_lang kana \
  --train_file $DIR/train.jsonl \
  --valid_file $DIR/valid.jsonl


num_epochs:50
Epoch: 1, Train loss: 3.289, Val loss: 2.594, Epoch time = 6228.494s
Epoch: 2, Train loss: 2.473, Val loss: 1.974, Epoch time = 6226.783s
Epoch: 3, Train loss: 2.082, Val loss: 1.652, Epoch time = 6220.123s
Epoch: 4, Train loss: 1.823, Val loss: 1.380, Epoch time = 6205.496s
Epoch: 5, Train loss: 1.603, Val loss: 1.179, Epoch time = 6208.515s
Epoch: 6, Train loss: 1.433, Val loss: 1.038, Epoch time = 6207.288s
Epoch: 7, Train loss: 1.305, Val loss: 0.929, Epoch time = 6197.311s
Epoch: 8, Train loss: 1.204, Val loss: 0.841, Epoch time = 6192.488s
Epoch: 9, Train loss: 1.122, Val loss: 0.775, Epoch time = 6188.396s
Epoch: 10, Train loss: 1.052, Val loss: 0.719, Epoch time = 6190.593s
Epoch: 11, Train loss: 0.991, Val loss: 0.670, Epoch time = 6199.745s
Epoch: 12, Train loss: 0.936, Val loss: 0.623, Epoch time = 6195.610s
Epoch: 13, Train loss: 0.888, Val loss: 0.587, Epoch time = 6195.088s
Epoch: 14, Train loss: 0.846, Val loss: 0.553, Epoch time = 6193.911s
Epoch: 15, Trai

In [12]:
%load_ext tensorboard
%tensorboard --logdir logs

## モデルを検証する
作成したモデルを，検証用データを用いて検証し，正解率を算出する


## パラメタ

- test_file
jsonl形式の検証用データファイル

- model_file
transformer_model.pyで作成したモデルファイル。

- outfile
検証結果を格納するファイル

- device
cpu限定

-nbest
検証用データの漢字姓名を入力データとした時に，モデルが出力するカタカナ姓名を確率の高い方からnbest個出力する。beam_width以下。searchパラメタがbothかbeamの時有効

- beam_width
ビームサーチのビーム幅。searchパラメタがbothかbeamの時有効

- max_len
出力するカタカナ姓名の最大長さ

- search
greedy,beam,bothから選択
-- greedy
貪欲法による探索
-- beam
ビームサーチによる探索
-- both
貪欲砲，ビームサーチ両方の探索



In [13]:
# 検証用データを実行


!python generate_batch.py \
  --test_file $DIR/test.jsonl \
  --model_file $MODEL/checkpoint_best.pt \
  --outfile $DIR/generate.txt \
  --device cpu \
  --nbest 5 \
  --beam_width 5 \
  --max_len 100 \
  --search both





Traceback (most recent call last):
  File "/Users/utsubo-katsuhiko/Documents/GitHub/kanjikana-model/train/generate_batch.py", line 18, in <module>
    from training.model import KanjiKanaTransformer, KanjiKanaDataSet, EOS_IDX, BOS_IDX, SPECIAL_SYMBOLS, engfra_tokenizer
ModuleNotFoundError: No module named 'training'


In [14]:
# 検証結果を評価
!python evaluate.py --infile $DIR/generate.txt



Traceback (most recent call last):
  File "/Users/utsubo-katsuhiko/Documents/GitHub/kanjikana-model/train/evaluate.py", line 54, in <module>
    main()
  File "/Users/utsubo-katsuhiko/Documents/GitHub/kanjikana-model/train/evaluate.py", line 51, in main
    run(args)
  File "/Users/utsubo-katsuhiko/Documents/GitHub/kanjikana-model/train/evaluate.py", line 45, in run
    print(f'{args.infile},acc={len(ok_prob)/len(hsh)}')
                               ~~~~~~~~~~~~^~~~~~~~~
ZeroDivisionError: division by zero


# モデルをJavaのDJL用に変換する
Pytorchで学習したモデルをJavaで使えるようにモデルの変換を行う。



In [15]:
# 漢字からカナ
!python convert_jitscript.py  \
    --model_file=$MODEL/checkpoint_best.pt \
    --model_script=$MODEL/script.pt \
    --encoder=$MODEL/encoder.pt \
    --decoder=$MODEL/decoder.pt \
    --positional_encoding=$MODEL/positional_encoding.pt \
    --generator=$MODEL/generator.pt \
    --src_tok_emb=$MODEL/src_tok_emb.pt \
    --tgt_tok_emb=$MODEL/tgt_tok_emb.pt \
    --vocab_src=$MODEL/vocab_src.txt \
    --vocab_tgt=$MODEL/vocab_tgt.txt \
    --params=$MODEL/params.json \
    --device=cpu



Traceback (most recent call last):
  File "/Users/utsubo-katsuhiko/Documents/GitHub/kanjikana-model/train/convert_jitscript.py", line 18, in <module>
    from train.char_model import KanjiKanaTransformer, KanjiKanaDataSet, EOS_IDX, BOS_IDX, SPECIAL_SYMBOLS
ModuleNotFoundError: No module named 'train'
